https://github.com/pytorch/examples/blob/master/dcgan/main.py

# DCGAN

In [1]:
from __future__ import print_function
#import argparse
import os
import random
import numpy as np

#import torch

import torch.multiprocessing as mp
mp.set_start_method('spawn')

import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable, grad

import matplotlib.pyplot as plt
%matplotlib inline

import h5py

import itertools
from matplotlib.ticker import MultipleLocator
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.ticker import MultipleLocator
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors
import matplotlib as mpl
import timeit

import seaborn as sns

ngpu = 1   

from power_spectrum_utils import power_spectrum_np
from wgan_utils import define_test, check_coords, get_samples
#from data_utils import get_max_cube, get_min_cube, get_mean_cube, get_stddev_cube
from plot_utils import hist_plot, plot_loss, plot_means

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#mean_5=get_mean_cube('../../../../../','5.0')
#std_5=get_stddev_cube('../../../../../','5.0',mean_5)
mean_5=14592.24
#mean_5=0
std_5=922711.56
max_5=4376932000
mask_value=10**2

random.seed(1)

In [3]:
class HydrogenDataset(Dataset):
    """Hydrogen Dataset"""

    def __init__(self, h5_file, s_test, s_sample, nsamples,transform, rotate):
        """
        Args:
            h5_file (string): name of the h5 file with 32 sampled cubes.
            root_dir (string): Directory with the .h5 file.
        """
        #file_size = os.path.getsize(root_dir + h5_file) / 1e6 # in MBs
        #print("The whole file size is " + str(int(file_size)) + " MBs")
        # self.subcubes = h5py.File('../data/sample_32.h5', 'r')
        self.s_test = s_test
        self.s_sample = s_sample
        self.t_coords = define_test(self.s_test,self.s_sample)
        self.nsamples = nsamples
        self.redshift = redshift
        self.transform=transform
        self.rotate=rotate
        self.file=h5_file

    def __len__(self):
        # Function called when len(self) is executed
        return self.nsamples

    def __getitem__(self, idx):
        """
        This can be implemented in such a way that the whole h5 file read 
        using h5py.File() and get_sample() function is called to return
        a random subcube. This won't increase memory usage because the
        subcubes will be read in the same way and only the batch will
        be read into memory.
        
        Here we have implemented it so that it can be used with data
        generated by get_sample() function.
        
        The output of this function is one subcube with the dimensions
        specified by get_sample() implementation.
        """
        
        sample = get_samples(file = self.file,
                            s_sample = self.s_sample,
                             nsamples = 1,
                             test_coords = self.t_coords)
        sample=sample.mean(axis=2)
        
        if self.rotate==True:
            t=random.randint(1,4)
            if t==4:
                sample=sample
            else:
                sample=np.rot90(sample, t, random.choice([(0,1), (1,0)]))
        
        if self.transform=='max':
            sample = sample / max_5
        if self.transform=='normalize':
            sample = (sample-mean_5)/std_5
        if self.transform=='clip_log':
            sample[sample <10**2]==0
            sample[sample > 3*std_5]==3*std_5
            sample=np.log(sample+1)
        
            
            
        if self.transform=='log':
            sample= np.log(sample+1)
        #print(np.array(sample).shape)
    
        sample=sample.reshape((1, self.s_sample, self.s_sample))

        return torch.tensor(sample)

In [4]:
def visualize2d(real, fake, save=''):
    max_=np.log(2.5*(10**6))
    #cols=real.shape[0]//2
    cols=8
    rows=2
    
    fig, axes = plt.subplots(nrows=2, ncols=cols, figsize=(16,4))
    
    for ax, row in zip(axes[:,0], ['Generated', 'Real']):
        ax.set_ylabel(row, rotation=90, fontsize=16)
    
    m=0
    for ax in axes.flat:
        #Plot only half of the mini-batch
        if m<cols:
            im = ax.imshow(np.log(fake[m][0]), aspect='equal', 
                       interpolation=None, vmin=0, vmax=max_)
        else:
            im = ax.imshow(np.log(real[m][0]), aspect='equal', 
                       interpolation=None, vmin=0, vmax=max_)
        m+=1
        ax.set_xticks([])
        ax.set_yticks([])
    #fig.subplots_adjust(right=.8)
    #cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    #fig.colorbar(ax, cax=cbar_ax)
    fig.tight_layout() 
    if save!='':
        fig.savefig(save+'samples.png', dpi=1000)
           
    plt.show()  

In [ ]:
 map_size = self.output_size/int(2**self.ng_layers)

In [14]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.Linear(nz, nc*imageSize*imageSize),
            nn.view(-1, imageSize, imageSize, nc )
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),
            
            
            nn.ConvTranspose2d(ngf*4 , ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(    ngf,      nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output

In [6]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)


In [7]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [15]:
#https://medium.com/@utk.is.here/
#keep-calm-and-train-a-gan-pitfalls-and-tips-on-training-generative-adversarial-networks-edd529764aa9
redshift='5.0'

imageSize=64
n_samples =3200
s_sample=imageSize
batchSize = 32

ngpu=1
nz=64
nc=1

ngf=32
ndf=32
cuda=True
device = torch.device("cuda:0")
#device =  torch.device("cuda")

lr = 0.0001
lrD =lr
lrG = lr
beta1=0.5

niter=50
giter=3
outf='first_iteration/'

datapath='../../../../../'
f = h5py.File(datapath+'fields_z='+redshift+'.hdf5', 'r')
f=f['delta_HI']
#"first_iteration/saved/netG_epoch_3.pth"  
netG_arg = ""  
netD_arg = ""  

    
dataset = HydrogenDataset( h5_file=f,
                            s_test = 1024, 
                            s_sample = s_sample, 
                            nsamples = n_samples, 
                            transform='normalize',
                            rotate=False)

workers=0
dataloader = torch.utils.data.DataLoader(dataset, 
                                         #batch_size=opt.batchSize,
                                         batch_size = batchSize,
                                         shuffle=True, 
                                         #num_workers=int(opt.workers)
                                         num_workers=int(workers) )



netG = Generator(ngpu).to(device)
netG.apply(weights_init)
if netG_arg != '':
    netG.load_state_dict(torch.load(netG_arg))
print(netG)

netD = Discriminator(ngpu).to(device)
netD.apply(weights_init)
if netD_arg != '':
    netD.load_state_dict(torch.load(netD_arg))
print(netD)

Generator(
  (main): Sequential(
    (0): Linear(in_features=64, out_features=128, bias=True)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(128, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [16]:
# torch.cuda.is_available()
criterion = nn.BCELoss()
if cuda:
    netG.cuda()
    netD.cuda()

fixed_noise = torch.randn(batchSize, nz, 1, 1, device=device)
real_label = 1
fake_label = 0

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

for epoch in range(niter):
    
    data_iter = iter(dataloader)
    i = 0
    while i < len(dataloader):
        data=data_iter.next()
        real_cpu = data.to(device)
    
    
    #for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD.zero_grad()
        #real_cpu = data[0].to(device)
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,), real_label, device=device)

        output = netD(real_cpu)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        G_iter=giter if  D_G_z1<.01 else 1
        
        
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()
        i+=1
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, niter, i, len(dataloader),
                 errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        if i%25 == 0:
            fake=netG(fixed_noise).detach()
            visualize2d(np.array(real_cpu)*std_5+mean_5, np.array(fake)*std_5+mean_5 , save='')
  

    # do checkpointing
    torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outf, epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (outf, epoch))    

RuntimeError: size mismatch, m1: [2048 x 1], m2: [64 x 128] at c:\new-builder_3\win-wheel\pytorch\aten\src\thc\generic/THCTensorMathBlas.cu:249